In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [3]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [4]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [37]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\TABLES_gender/marburg_virus_data_clean_concat_and_indicator_2007.csv')#.drop(['Addresses'], axis=1)
data.head(2)

,Unnamed: 0,wos_ID,Funding Name Preferred,Open Access Designations,Publisher,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year,Affiliations,...,wb_low_income_economies_africa,wb_low_income_economies_others,wb_lower_middle_income_economies,wb_lower_middle_income_economies_africa,wb_lower_middle_income_economies_others,wb_upper_middle_income_economies,wb_upper_middle_income_economies_africa,wb_upper_middle_income_economies_others,wb_high_income_economies,wb_high_income_economies_others
0,203,wos:000243562000002,NaN,NaN,w b saunders co ltd,10,1,11,2007.0,university of london; london school of hygiene...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
1,204,wos:000259305000001,NaN,NaN,springer india,2,1,15,2007.0,sher-e-kashmir university of agricultural scie...,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [10]:
data.count().head(10)

Unnamed: 0                    714
wos_ID                        714
Funding Name Preferred        561
Open Access Designations      581
Publisher                     714
Times Cited, All Databases    714
180 Day Usage Count           714
Since 2013 Usage Count        714
Publication Year              714
Affiliations                  668
dtype: int64

# The accessibility of Marburg Virus Diseases publications (open access)

## proportion all

In [11]:
countries_oa_list_all = data.reset_index().groupby(['open_access_yes']).count().T
countries_oa_list_all.rename(columns = {0:'all_is_no_oa', 
                              1:'all_is_oa'}, inplace = True)

countries_oa_list_all['all_total'] = countries_oa_list_all['all_is_no_oa'] + countries_oa_list_all['all_is_oa']

for col in countries_oa_list_all.columns:
    countries_oa_list_all['%'+col] = countries_oa_list_all[col]/countries_oa_list_all['all_total']*100
    
countries_oa_list_all['%_diff(%all_is_oa-%all_is_no_oa)'] = countries_oa_list_all['%all_is_oa'] - countries_oa_list_all['%all_is_no_oa']

# save data
# countries_oa_list_all.to_csv(f'../TABLES/countries_all_oa_list.csv')

countries_oa_list_all

open_access_yes,all_is_no_oa,all_is_oa,all_total,%all_is_no_oa,%all_is_oa,%all_total,%_diff(%all_is_oa-%all_is_no_oa)
index,133,581,714,18.627451,81.372549,100.0,62.745098
Unnamed: 0,133,581,714,18.627451,81.372549,100.0,62.745098
wos_ID,133,581,714,18.627451,81.372549,100.0,62.745098
Funding Name Preferred,74,487,561,13.190731,86.809269,100.0,73.618538
Open Access Designations,0,581,581,0.000000,100.000000,100.0,100.000000
...,...,...,...,...,...,...,...
wb_upper_middle_income_economies,24,72,96,25.000000,75.000000,100.0,50.000000
wb_upper_middle_income_economies_africa,4,22,26,15.384615,84.615385,100.0,69.230769
wb_upper_middle_income_economies_others,21,56,77,27.272727,72.727273,100.0,45.454545
wb_high_income_economies,112,553,665,16.842105,83.157895,100.0,66.315789


In [12]:
countries_oa_list_all[['all_is_no_oa','all_is_oa','all_total']].describe()

open_access_yes,all_is_no_oa,all_is_oa,all_total
count,146.000000,146.000000,146.000000
mean,35.671233,165.157534,200.828767
std,55.129704,249.719531,303.808024
min,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000
50%,2.000000,9.000000,12.500000
75%,76.250000,530.500000,576.000000
max,133.000000,581.000000,714.000000


### significance

In [54]:
# first_ authors
first_male_OA = data.loc[(data['first_author_gender'] == 'male') & (data['number_of_author'] >1), 'open_access_yes'].to_list()
first_female_OA = data.loc[(data['first_author_gender'] == 'female') & (data['number_of_author'] >1), 'open_access_yes'].to_list()

dfs = [first_male_OA,first_female_OA]
for df in dfs:
    if len(df)>=3 :
        test_si = test_sig(first_male_OA, first_female_OA)
test_si

mann-whitney
mann-whitney


MannwhitneyuResult(statistic=52904.5, pvalue=0.44814507670080495)

In [55]:
# last authors
last_male_OA = data.loc[(data['last_author_gender'] == 'male') & (data['number_of_author'] >1), 'open_access_yes'].to_list()
last_female_OA = data.loc[(data['last_author_gender'] == 'female') & (data['number_of_author'] >1), 'open_access_yes'].to_list()

dfs = [last_male_OA,last_female_OA]
for df in dfs:
    if len(df)>=3 :
        test_si = test_sig(last_male_OA, last_female_OA)
test_si

mann-whitney
mann-whitney


MannwhitneyuResult(statistic=34978.0, pvalue=0.5747394717501244)

In [63]:
# solo authors
solo_male_OA = data.loc[(data['last_author_gender'] == 'male') & (data['number_of_author'] == 1), 'open_access_yes'].to_list()
solo_female_OA = data.loc[(data['last_author_gender'] == 'female') & (data['number_of_author'] == 1), 'open_access_yes'].to_list()

dfs = [solo_male_OA,solo_female_OA]
for df in dfs:
    if len(df)>=3:
        test_sig(solo_male_OA, solo_female_OA)

mann-whitney
mann-whitney


In [61]:
solo_female_OA

[0, 0, 1, 1, 1, 1]

In [59]:
len(solo_female_OA)

6

In [62]:
solo_male_OA

[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1]

In [60]:
len(solo_male_OA)

21

## proportion solo

In [64]:
data_solo = data.loc[data['number_of_author'] == 1]
data_solo.shape

(27, 146)

In [65]:
data_solo['number_of_author'].value_counts()

1.0    27
Name: number_of_author, dtype: int64

In [68]:
countries_oa_list_gender = data_solo.reset_index().groupby(['open_access_yes']).count().T
countries_oa_list_gender.rename(columns = {0:'gender_is_no_oa', 
                              1:'gender_is_oa'}, inplace = True)

countries_oa_list_gender['gender_total'] = countries_oa_list_gender['gender_is_no_oa'] + countries_oa_list_gender['gender_is_oa']

for col in countries_oa_list_gender.columns:
    countries_oa_list_gender['%'+col] = countries_oa_list_gender[col]/countries_oa_list_gender['gender_total']*100
    
countries_oa_list_gender['%_diff(%gender_is_oa-%gender_is_no_oa)'] = countries_oa_list_gender['%gender_is_oa'] - countries_oa_list_gender['%gender_is_no_oa']

# save data
# countries_oa_list_fund.to_csv(f'../TABLES/countries_fund_oa_list.csv')

countries_oa_list_gender

open_access_yes,gender_is_no_oa,gender_is_oa,gender_total,%gender_is_no_oa,%gender_is_oa,%gender_total,%_diff(%gender_is_oa-%gender_is_no_oa)
index,12,15,27,44.444444,55.555556,100.0,11.111111
Unnamed: 0,12,15,27,44.444444,55.555556,100.0,11.111111
wos_ID,12,15,27,44.444444,55.555556,100.0,11.111111
Funding Name Preferred,4,10,14,28.571429,71.428571,100.0,42.857143
Open Access Designations,0,15,15,0.000000,100.000000,100.0,100.000000
...,...,...,...,...,...,...,...
wb_upper_middle_income_economies,1,2,3,33.333333,66.666667,100.0,33.333333
wb_upper_middle_income_economies_africa,0,0,0,NaN,NaN,NaN,NaN
wb_upper_middle_income_economies_others,1,2,3,33.333333,66.666667,100.0,33.333333
wb_high_income_economies,11,11,22,50.000000,50.000000,100.0,0.000000


## proportion no fund

In [12]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund.head(2)

,wos_ID,Addresses,Afghanistan,United States,Angola,Argentina,Australia,Austria,Bangladesh,Belgium,...,wb_lower_middle_income_economies,wb_lower_middle_income_economies_africa,wb_lower_middle_income_economies_others,wb_upper_middle_income_economies,wb_upper_middle_income_economies_africa,wb_upper_middle_income_economies_others,wb_high_income_economies,wb_high_income_economies_others,funding_yes,open_access_yes
0,wos:000904661800001,"[Bi, Jinhao; Xia, Xianzhu] Jilin Agr Univ, Col...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,0,1
4,wos:000849602200001,"[Sah, Ranjit] Tribhuvan Univ Teaching Hosp, In...",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,2.0,NaN,NaN,NaN,1.0,1.0,0,1


In [13]:
countries_oa_list_no_fund = data_no_fund.reset_index().groupby(['open_access_yes']).count().T
countries_oa_list_no_fund.rename(columns = {0:'no_fund_is_no_oa', 
                              1:'no_fund_is_oa'}, inplace = True)

countries_oa_list_no_fund['no_fund_total'] = countries_oa_list_no_fund['no_fund_is_no_oa'] + countries_oa_list_no_fund['no_fund_is_oa']

for col in countries_oa_list_no_fund.columns:
    countries_oa_list_no_fund['%'+col] = countries_oa_list_no_fund[col]/countries_oa_list_no_fund['no_fund_total']*100
    
countries_oa_list_no_fund['%_diff(%no_fund_is_oa-%no_fund_is_no_oa)'] = countries_oa_list_no_fund['%no_fund_is_oa'] - countries_oa_list_no_fund['%no_fund_is_no_oa']

# save data
# countries_oa_list_no_fund.to_csv(f'../TABLES/countries_fund_no_oa_list.csv')

countries_oa_list_no_fund

open_access_yes,no_fund_is_no_oa,no_fund_is_oa,no_fund_total,%no_fund_is_no_oa,%no_fund_is_oa,%no_fund_total,%_diff(%no_fund_is_oa-%no_fund_is_no_oa)
index,181,174,355,50.985915,49.014085,100.0,-1.971831
wos_ID,181,174,355,50.985915,49.014085,100.0,-1.971831
Addresses,132,167,299,44.147157,55.852843,100.0,11.705686
Afghanistan,0,1,1,0.000000,100.000000,100.0,100.000000
United States,64,88,152,42.105263,57.894737,100.0,15.789474
...,...,...,...,...,...,...,...
wb_upper_middle_income_economies_africa,6,13,19,31.578947,68.421053,100.0,36.842105
wb_upper_middle_income_economies_others,14,7,21,66.666667,33.333333,100.0,-33.333333
wb_high_income_economies,118,156,274,43.065693,56.934307,100.0,13.868613
wb_high_income_economies_others,118,156,274,43.065693,56.934307,100.0,13.868613


## concat all dataframe and save

In [14]:
# concatenate dataframes on index
concat_countries_oa_list = pd.concat([countries_oa_list_all, 
                                        countries_oa_list_fund, 
                                        countries_oa_list_no_fund], 
                                       axis=1)
# save data
concat_countries_oa_list.to_csv(f'../TABLES/concat_countries_oa_list.csv')
concat_countries_oa_list

open_access_yes,all_is_no_oa,all_is_oa,all_total,%all_is_no_oa,%all_is_oa,%all_total,%_diff(%all_is_oa-%all_is_no_oa),fund_is_no_oa,fund_is_oa,fund_total,...,%fund_is_oa,%fund_total,%_diff(%fund_is_oa-%fund_is_no_oa),no_fund_is_no_oa,no_fund_is_oa,no_fund_total,%no_fund_is_no_oa,%no_fund_is_oa,%no_fund_total,%_diff(%no_fund_is_oa-%no_fund_is_no_oa)
index,259,673,932,27.789700,72.210300,100.0,44.420601,78,499,577,...,86.481802,100.0,72.963605,181,174,355,50.985915,49.014085,100.0,-1.971831
wos_ID,259,673,932,27.789700,72.210300,100.0,44.420601,78,499,577,...,86.481802,100.0,72.963605,181,174,355,50.985915,49.014085,100.0,-1.971831
Addresses,210,666,876,23.972603,76.027397,100.0,52.054795,78,499,577,...,86.481802,100.0,72.963605,132,167,299,44.147157,55.852843,100.0,11.705686
Afghanistan,0,1,1,0.000000,100.000000,100.0,100.000000,0,0,0,...,NaN,NaN,NaN,0,1,1,0.000000,100.000000,100.0,100.000000
United States,110,476,586,18.771331,81.228669,100.0,62.457338,46,388,434,...,89.400922,100.0,78.801843,64,88,152,42.105263,57.894737,100.0,15.789474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wb_upper_middle_income_economies_africa,8,30,38,21.052632,78.947368,100.0,57.894737,2,17,19,...,89.473684,100.0,78.947368,6,13,19,31.578947,68.421053,100.0,36.842105
wb_upper_middle_income_economies_others,27,60,87,31.034483,68.965517,100.0,37.931034,13,53,66,...,80.303030,100.0,60.606061,14,7,21,66.666667,33.333333,100.0,-33.333333
wb_high_income_economies,185,635,820,22.560976,77.439024,100.0,54.878049,67,479,546,...,87.728938,100.0,75.457875,118,156,274,43.065693,56.934307,100.0,13.868613
wb_high_income_economies_others,185,635,820,22.560976,77.439024,100.0,54.878049,67,479,546,...,87.728938,100.0,75.457875,118,156,274,43.065693,56.934307,100.0,13.868613
